In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
from IPython.display import clear_output
#set_matplotlib_formats('retina')
#from tqdm import tqdm
#tqdm.pandas()
import numpy as np
from pyhive import presto
from datetime import datetime, timedelta
#from bson import ObjectId
from functools import reduce
from sklearn.cluster import KMeans
#from pymongo import MongoClient
import glob
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import json
#import re
#import dtale
#from h3 import h3
# import pandasql as ps

In [3]:
metabase_connection = presto.connect(
        #host='bi-trino-2.serving.data.production.internal',
        #host='bi-trino.serving.data.production.internal',
        #host='bi-presto.serving.data.production.internal',
        #host='prime-trino.serving.data.production.internal',
        host="presto.processing.yoda.run",
        port=80,
        protocol='http',
        catalog ='hive',
        username='praveen.u@rapido.bike')

In [5]:
odd_even=pd.DataFrame()

for i in pd.date_range('20230510','20230523'):
    query = """

    with order_filtering as 
    (
        select distinct order_id ,city_name, 
                CASE 
                    WHEN substr(quarter_hour,1,2) IN ('08','09','10','11') THEN 'morning_peak'
                    WHEN substr(quarter_hour,1,2) IN ('12','13','14','15','16') THEN 'afternoon'
                    WHEN substr(quarter_hour,1,2) IN ('17','18','19','20','21') THEN 'evening_peak'
                    WHEN substr(quarter_hour,1,2) IN ('00','01','02','03','04','05','06','07') THEN 'rest_morning'        
                    ELSE 'rest_evening' END AS time_period, order_status

        from 
            orders.order_logs_snapshot
        where yyyymmdd = cast({dt} as varchar)
        --where yyyymmdd = '20230502'
            and service_obj_service_name = 'Auto'
            and (spd_fraud_flag = false OR spd_fraud_flag IS NULL)
            and lower(trim(city_name)) in ('hyderabad')
    )

    , rider_pings as
    (

            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
            from raw.kafka_dispatch_propagation_immutable
            where yyyymmdd = cast({dt} as varchar)
            --where yyyymmdd = '20230502'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_acknowledged')


            union all

            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,
            concat(data_propagationBatchId,data_riderId,data_orderid) as unique_ping_level_id,
            concat(data_orderid,data_riderId) as unique_id_order_level_id,epoch
            from(
            select eventtype,data_propagationBatchId,data_riderId,data_orderid,yyyymmdd,epoch,
            row_number() over(partition by data_propagationBatchId,data_riderId,data_orderid order by epoch desc) as row,data

            from raw.kafka_dispatch_propagation_immutable
            where yyyymmdd = cast({dt} as varchar)
            --where yyyymmdd = '20230502'

                and data_orderid in (select distinct order_id from order_filtering)
                and data_propagationtype in ('multicast')
                and eventtype in ('rider_busy','order_already_accepted','rider_accept_failed','rider_rejected','rider_accepted','rider_ineligible','rider_unreachable')
                ) base
            where row =1

    )
        ,order_screen as (
            select distinct order_id,userid as captain_id, yyyymmdd,scroll_type, --deviceInfo_model,deviceInfo_make,
            case  
                when length(profile_phone)!= 12 then 'blank'
                when (length(profile_phone) = 12) and cast((substr(profile_phone,-1)) as integer)%2 = 1 then 'odd'
                when (length(profile_phone) = 12) and cast((substr(profile_phone,-1)) as integer)%2 = 0 then 'even'
                else 'blank' 
            end as odd_even_flag,

            case
                when  deviceInfo_appVersion >= '5.2.17' then 'gt'
                else 'les' end as device_info,

            date_format(date_trunc('month', date_parse(yyyymmdd,'%Y%m%d')), '%M-%Y') as Month,order_rank, deviceinfo_appversion
            from (
                select *,
                concat(order_id,userId) as unique_id,
                ROW_NUMBER() OVER(PARTITION BY order_id,userid ORDER BY epoch) as rn
                    from(
                        select 
                            distinct yyyymmdd,profile_phone,
                            JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.order_id') as order_id,
                            JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.userId') as userid,
                            JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.order_rank') as order_rank,
                            JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.mmo_scroll') as scroll_type,
                            epoch, deviceinfo_appversion
                        from raw.clevertap_captain_events_master a 
                        where yyyymmdd = cast({dt} as varchar)
                        --where yyyymmdd = '20230502'
                        and eventname in  ('mmo_order_received') --('mmo_booking_response')
                        and JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.order_id') in (select distinct order_id from order_filtering)
                    )
                )
            where rn=1 

        )
        ,exit_screen as (

                 select distinct order_id,userid as captain_id, exit_rank,order_rank_1
                         from (
                        select *,
                        concat(order_id,userId) as unique_id,
                        ROW_NUMBER() OVER(PARTITION BY order_id,userid ORDER BY epoch) as rn
                            from(
                                select 
                                    distinct yyyymmdd,profile_phone,
                                    JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.order_id') as order_id,
                                    JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.userId') as userid,
                                    JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.order_rank') as order_rank_1,
                                    JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.exit_rank') as exit_rank,
                                    epoch, deviceinfo_appversion
                                from raw.clevertap_captain_events_master a 
                                where yyyymmdd = cast({dt} as varchar)
                                --where yyyymmdd = '20230502'
                                and eventname in  ('mmo_booking_response')
                                and JSON_EXTRACT_SCALAR(eventProps, '$.eventProps.order_id') in (select distinct order_id from order_filtering)
                            )
                        )
                    where rn=1 
        )
        ,
        ord_immutable as 
        (    
        select order_id, captain_id, sum(case when order_status = 'dropped' then 1 else 0 end) as order_status_Flag
            from orders.order_logs_immutable
            where yyyymmdd = cast({dt} as varchar)
            --where yyyymmdd = '20230502'
            and captain_id is not NULL
            and captain_id != ''
            and order_id in (select distinct order_id from order_filtering)
            GROUP BY 1,2
    ),
   seg as  (
    select captain_id, yyyymmdd, performance_segment,consistency,concat(performance_segment,'_',consistency) as pc_segment 
                        from datasets_internal.captain_auto_pc_segments_v1
                        where yyyymmdd = cast({dt} as varchar)
                        and lower(trim(city_name)) in ('hyderabad')
    )

    select yyyymmdd,city_name ,time_period, order_screen,order_screen_1,exit_screen,
    odd_even_flag,device_info,scroll_type,performance_segment,consistency,pc_segment,

    count(distinct case when eventtype='rider_busy' then unique_ping_level_id end) as riderBusy_pings,
    count(distinct case when eventtype='rider_rejected' then unique_ping_level_id end) as riderreject_Pings,
    count(distinct case when eventtype='rider_accepted' then unique_ping_level_id end) as accepted_pings,
    count(distinct case when eventtype='order_already_accepted' then unique_ping_level_id end) as already_accepted_pings,
    count(distinct case when eventtype='rider_accept_failed' then unique_ping_level_id end) as already_accepted_intent_pings,
    count(distinct case when eventtype='rider_acknowledged' then unique_ping_level_id end) as total_pings


    from(

        Select distinct a.* ,
        b.order_status_Flag ,

        f.city_name, f.time_period,
        g.order_rank as order_screen,
        h.order_rank_1 as order_screen_1,
        h.exit_rank as exit_screen,
        g.odd_even_flag,
        g.device_info,
        g.scroll_type,
        s.performance_segment,
        s.consistency,
        s.pc_segment

        from rider_pings  a 
        left join ord_immutable b on a.data_riderId = b.captain_id and a.data_orderid = b.order_id


        left join order_filtering f on a.data_orderid = f.order_id

        left join order_screen g on a.data_orderid = g.order_id and a.data_riderId = g.captain_id
        left join exit_screen h on a.data_orderid = h.order_id and a.data_riderId = h.captain_id
        
        left join seg as s on a.data_riderId= s.captain_id and a.yyyymmdd= s.yyyymmdd
    )
    group by 1,2,3,4,5,6,7,8,9,10,11,12
    """.format(dt=i.strftime('%Y%m%d'))

    df_test = pd.read_sql_query(query, metabase_connection)
    
    print(i)
    odd_even=odd_even.append(df_test)
    

2023-05-10 00:00:00
2023-05-11 00:00:00
2023-05-12 00:00:00
2023-05-13 00:00:00
2023-05-14 00:00:00
2023-05-15 00:00:00
2023-05-16 00:00:00
2023-05-17 00:00:00
2023-05-18 00:00:00
2023-05-19 00:00:00
2023-05-20 00:00:00
2023-05-21 00:00:00
2023-05-22 00:00:00
2023-05-23 00:00:00


In [6]:
odd_even

,yyyymmdd,city_name,time_period,order_screen,order_screen_1,exit_screen,odd_even_flag,device_info,scroll_type,performance_segment,consistency,pc_segment,riderBusy_pings,riderreject_Pings,accepted_pings,already_accepted_pings,already_accepted_intent_pings,total_pings
0,20230510,Hyderabad,afternoon,1.0,2.0,1.0,even,gt,vertical,MP,Daily,MP_Daily,145,46,6,45,5,261
1,20230510,Hyderabad,evening_peak,1.0,1.0,None,blank,gt,carousel,LP,intra,LP_intra,454,1001,75,407,25,2011
2,20230510,Hyderabad,morning_peak,3.0,3.0,1.0,even,les,vertical,LP,Daily,LP_Daily,443,73,4,44,0,585
3,20230510,Hyderabad,afternoon,2.0,2.0,2.0,even,les,vertical,LP,intra,LP_intra,57,106,14,464,0,693
4,20230510,Hyderabad,afternoon,2.0,None,None,even,gt,vertical,LP,inter,LP_inter,9517,8,0,360,69,10321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301,20230523,Hyderabad,morning_peak,3.0,1.0,None,odd,les,carousel,MP,Daily,MP_Daily,5,4,0,1,0,10
4302,20230523,Hyderabad,rest_morning,1.0,3.0,3.0,even,gt,vertical,None,None,None,1,0,0,0,0,2
4303,20230523,Hyderabad,evening_peak,4.0,3.0,3.0,even,gt,vertical,MP,inter,MP_inter,1,0,0,1,0,2
4304,20230523,Hyderabad,evening_peak,3.0,2.0,None,blank,gt,carousel,LP,Daily,LP_Daily,4,0,0,0,0,4


In [7]:
odd_even.to_clipboard()

In [30]:
#df_test.to_csv("MMO_OA_v3.csv", index=False)